### Imports

In [7]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

### Get Article Links

In [35]:
def get_links(start, stop):
    titles = []
    urls=[]
    for i in range(start, stop+1):
        print(i)
        ua = UserAgent()
        user_agent = {'User-agent': ua.random}
        url = f'https://www.nature.com/nature/articles?searchType=journalSearch&sort=PubDate&type=news&page={i}'
        response = requests.get(url, headers = user_agent)
        page = response.text
        soup = BeautifulSoup(page, 'lxml')
        page_articles = soup.findAll('article')
        title_list = []
        link_list=[]
        for article in page_articles:
            title_list.append(article.find('a').text.strip())
            link_list.append(article.find('a')['href'])
        titles += title_list
        urls += link_list
    return pd.DataFrame(data={'title':titles, 'url':urls})
        
        
        

In [2]:
# df = get_links(1, 1000)

### Use links to get article text, and date published

In [5]:
df['comment_text'] = ['' for i in range(len(df.index))]
df['year'] = [0 for i in range(len(df.index))]
df['month'] = ['' for i in range(len(df.index))]
df['day'] = [0 for i in range(len(df.index))]

In [89]:
base_url = 'https://www.nature.com'
for i, url in enumerate(new_df.url):
    if i > 11201:
        print(i)
        full_url = base_url + url
        ua = UserAgent()
        user_agent = {'User-agent': ua.random}
        try:
            response = requests.get(full_url, headers = user_agent, allow_redirects=True)
            page = response.text
            soup = BeautifulSoup(page, 'lxml')
        except:
            new_df.loc[i, 'comment_text'] = 'Unknown'


        article_text = ''
        article_body=''
        
        if soup.find('div', class_='article__body serif cleared'):
            article_body = soup.find('div', class_='article__body serif cleared')
        elif soup.find('div', class_='entry-content'):
            article_body = soup.find('div', class_='entry-content')
        elif soup.find('article'):
            article_body = soup.find('article')
        else:
            article_text= 'Unknown'
            print('Unknown')

        if article_body:
            paragraph_list = article_body.findAll('p', class_ = False)
            for paragraph in paragraph_list:
                if paragraph == paragraph_list[-1]:
                    article_text += paragraph.text.strip()
                else:
                    article_text += (paragraph.text.strip() + ' ')


        new_df.loc[i, 'comment_text'] = article_text

        if soup.find('time'):
            time = soup.find('time').text.split(' ')
            new_df.loc[i, 'day'] = int(time[0])
            new_df.loc[i, 'month'] = time[1]
            new_df.loc[i, 'year'] = int(time[2])
        elif soup.find('abbr', class_='published'):
            time = soup.find('abbr').text.split(' ')
            new_df.loc[i, 'day'] = int(time[0])
            new_df.loc[i, 'month'] = time[1]
            new_df.loc[i, 'year'] = int(time[2])